## Configs

In [ ]:
import random

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_colwidth', None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

pd.options.plotting.backend = "plotly"
pd.options.plotting.backend = "matplotlib"

random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Dataframe

In [2]:

import pandas as pd

dfs_to_concat = []

df = (
    pd.read_csv(
        "file_name.csv",
        header="infer",
        sep=";",
        encoding="None",
        on_bad_lines="skip",
        usecols=["col_1", "col_2"],
        thousands=None,
        decimal="."
    )
    .head()
    .iloc[:-3, :-1]
    .query("date < 20200101")
    .query("name == 'cel_name'")
    .query("`name with space` == 'cel_name'")
    .query("name == @my_variable")
    .query("col.str.contains('cel_name', case=True)")
    .reset_index(drop=False)
    .set_index(["index_1_col", "index_2_col"])
    .melt(id_vars=["col_1"])
    .pivot_table(index="col_1", columns="col_2", values="col_3", aggfunc=np.sum, fill_value=0)
    .explode("col")
    .value_counts(ascending=False, normalize=False)
    .to_frame()
    .droplevel(0, axis=1)
    .unstack()
    .sort_index()
    .assign(date = lambda df: pd.to_datetime(df["date"], format="%d/%m/%Y", errors="coerce" ))
    .assign(new_col = lambda df: df["col"].apply(
        lambda row: len(row))
    )
    .sort_values(["date"])
    .join(df, how="outer")
    .resample("1D")
    .groupby(pd.Grouper(key="date", freq="1W"))
    .mean()
    .ffill()
    .fillna(0)
    .add_prefix("my_prefix_")
    .rename(
        columns={
            "old_name": "new_name",
        }
    )
    .drop(
        [
            "col_to_drop",
        ],
        axis=1
    )
    .drop_duplicates()
    .astype(int)
    [["col_1", "col_3"]]
    .tail()
)

dfs_to_concat.append(df_to_concat)

dfs_concatenated = pd.concat(dfs_to_concat, axis=0, ignore_index=True)

## Plot

In [ ]:
import plotly.graph_objs as go

fig = go.Figure()

# https://plotly.com/python/reference/scatter/
fig.add_trace(go.Scatter(
    x=df["date_col"],
    y=df["another_col_1"],
    name="Line Name",
    line={
        "width": 5,
        "dash": "dash",
        "color": "black"
    }
))

fig.add_trace(go.Scatter(
    x=df["date_col"],
    y=df["another_col_2"],
    name="Line Name",
    line={
        "width": 5,
        "dash": "dot",
        "color": "green"
    }
))

# https://plotly.com/python/reference/layout/
fig.update_layout({
    "title": {
        "text": f"Title",
        "font": {
            "size": 25
        },
        "x": 0.5
    },
    "font": {
        "size": 20
    },
    "height": 600
})

fig.show()